# 데이터 스플릿해주는 코드는 나중에 구현

# 파이프라인 진행

In [56]:
# 라이브러리 및 모듈을 불러옴.
import numpy as np
import pandas as pd
import time
import timeit
from datetime import datetime
import os
import glob
import sys
import matplotlib.pyplot as plt

# matplotlib의 기본 이미지 컬러맵을 'gray'로 설정.
plt.rcParams['image.cmap'] = 'gray'

# 이미지 처리를 위한 OpenCV와 PIL 라이브러리를 불러옴.
import cv2
from PIL import Image

# 난수 생성을 위한 random 모듈과 객체 복사를 위한 copy 모듈을 불러옴.
import random
import copy

# 경고 메시지를 무시하도록 설정.
import warnings
warnings.filterwarnings('ignore')

# 현재 세션의 파일 이름을 가져와 FILENAME 변수에 저장.
FILENAME = os.getcwd()+'/'+str(__session__).split('/')[-1]

# PyTorch 관련 모듈을 불러옴.
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

# TorchVision을 사용한 데이터셋과 전처리 도구들을 불러옴.
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import RandomResizedCrop
import torchvision.transforms.functional as TF

# 데이터셋 및 데이터 로더를 위한 유틸리티를 불러옴.
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# 데이터 분할을 위한 Scikit-Learn의 유틸리티를 불러옴.
from sklearn.model_selection import train_test_split, KFold

# MONAI 라이브러리에서 손실 함수와 메트릭을 불러옴.
from monai.losses import TverskyLoss as TverskyLoss
from monai.transforms import Compose, ToTensor, RandFlip
from monai.metrics import DiceMetric as Dice_Function
from monai.metrics import compute_iou as IoU_Function
from monai.metrics import ConfusionMatrixMetric

# 시스템 모듈의 경로에 상위 디렉토리를 추가.
import sys
sys.path.append("..")

# 모델이 위치한 디렉토리를 지정.
model_dir = 'models'

# 모델 디렉토리에서 사용할 모듈 이름을 리스트로 지정. (ST, ST++, AugSeg, S4Former)
module_names = ['ST', 'ST++', 'AugSeg']
model_names = module_names

# 지정된 모듈을 동적으로 불러옴.
for module_name in module_names:
    exec(f'from {model_dir}.{module_name} import *')

# 반복 횟수를 지정. (추후 데이터 스플릿한 폴더 지정하여 반복실험 가능하게끔)
iterations = [1, 10]

# 입력 채널 수와 클래스 수를 설정.
in_channels = 3
number_of_classes = 1

# 학습에 사용할 에포크 수를 설정.
epochs = 80  # PASCAL VOC 2012 기준: 80 epochs

# 얼리 스탑핑을 위한 기준 에포크 수를 설정.
EARLY_STOP = 25  

# 배치 크기를 설정.
batch_size = 16

# 사용할 GPU 장치 번호를 설정. (GALAX RTX 3090 EX GAMER)
devices = [0]

# 옵티마이저의 종류와 학습률, 모멘텀, 가중치 감소율을 설정.
backbone_lr = 1e-3
segmentation_head_lr = 1e-2  # 10배 큰 학습률
weight_decay = 1e-4
optim_args = {'backbone_lr': backbone_lr, 'segmentation_head_lr': segmentation_head_lr, 'weight_decay': weight_decay}

# 학습률 스케줄러로 Poly LR을 설정하고 관련 매개변수를 지정.
def poly_lr_scheduler(optimizer, init_lr, iter, max_iter, power=0.9):
    new_lr = init_lr * (1 - iter / max_iter) ** power
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr
    return new_lr

In [51]:
# 손실 함수로 Cross-Entropy Loss와 Consistency Regularization Loss를 포함하는 손실 함수를 설정
class CrossEntropyWithConsistencyLoss(nn.Module):
    def __init__(self, lambda_consistency=1.0):
        super(CrossEntropyWithConsistencyLoss, self).__init__()
        self.lambda_consistency = lambda_consistency
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, student_outputs, teacher_outputs, targets):
        ce_loss = self.ce_loss(student_outputs, targets)
        consistency_loss = torch.mean((student_outputs - teacher_outputs) ** 2)
        return ce_loss + self.lambda_consistency * consistency_loss

# 랜덤 시드를 설정하는 함수.
def control_random_seed(seed, pytorch=True):
    random.seed(seed)  # Python의 기본 random 모듈 시드 설정
    np.random.seed(seed)  # NumPy의 시드 설정
    try:
        torch.manual_seed(seed)  # PyTorch의 시드 설정
        if torch.cuda.is_available() == True:  # CUDA 사용 가능 여부 확인
            torch.cuda.manual_seed(seed)  # GPU의 시드 설정
            torch.cuda.manual_seed_all(seed)  # 모든 GPU에 대해 시드 설정
            torch.backends.cudnn.deterministic = True  # CUDA 연산을 결정론적으로 설정
            torch.backends.cudnn.benchmark = False  # 연산 속도 최적화를 비활성화
    except:
        pass
        torch.backends.cudnn.benchmark = False  # 예외 발생 시 연산 속도 최적화를 비활성화

# 파일 경로에서 이미지를 읽어오는 함수.
def imread_kor(filePath, mode=cv2.IMREAD_UNCHANGED):
    stream = open(filePath.encode("utf-8"), "rb")  # 파일을 바이너리 읽기 모드로 열기
    bytes = bytearray(stream.read())  # 파일 데이터를 바이트 배열로 읽기
    numpyArray = np.asarray(bytes, dtype=np.uint8)  # 바이트 배열을 NumPy 배열로 변환
    return cv2.imdecode(numpyArray, mode)  # NumPy 배열을 이미지로 디코딩하여 반환

# 파일 경로에 이미지를 저장하는 함수.
def imwrite_kor(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]  # 파일 확장자 추출
        result, n = cv2.imencode(ext, img, params)  # 이미지를 인코딩하여 바이트 배열로 변환
        if result:
            with open(filename, mode='w+b') as f:  # 바이너리 쓰기 모드로 파일 열기
                n.tofile(f)  # 바이트 배열을 파일에 쓰기
                return True
        else:
            return False
    except Exception as e:
        print(e)  # 예외 발생 시 오류 메시지 출력
        return False

# 이미지와 마스크를 주어진 각도 범위 내에서 무작위로 회전시키는 함수.
def random_rotation(image, mask, angle_range=(-30, 30)):
    angle = random.uniform(angle_range[0], angle_range[1])  # 주어진 범위에서 무작위 각도 선택
    image = TF.rotate(image, angle)  # 이미지 회전
    mask = TF.rotate(mask, angle)  # 마스크 회전
    return image, mask

# 이미지와 마스크 경로 리스트를 받아 데이터셋을 만드는 클래스.
class ImagesDataset(Dataset):
    def __init__(self, image_path_list, target_path_list, aug=False):
        self.image_path_list = image_path_list  # 이미지 경로 리스트
        self.target_path_list = target_path_list  # 마스크 경로 리스트
        self.transform = transforms.Compose([
            transforms.ToTensor(),  # 이미지를 텐서로 변환
        ])
        self.aug = aug  # 데이터 증강 여부 설정

    def __len__(self):
        return len(self.image_path_list)  # 데이터셋의 길이 반환

    def __getitem__(self, idx):
        image_path = self.image_path_list[idx]  # 인덱스에 해당하는 이미지 경로 가져오기
        mask_path = self.target_path_list[idx]  # 인덱스에 해당하는 마스크 경로 가져오기
        image = imread_kor(image_path)  # 이미지 읽기
        image = self.transform(image).float()  # 이미지 변환 및 float형 변환

        mask = imread_kor(mask_path)  # 마스크 읽기
        mask = self.transform(mask).float()  # 마스크 변환 및 float형 변환

        # 데이터 증강이 활성화된 경우, 추가적인 변환을 수행.
        if self.aug == True:
            if random.random() < 0.5:
                resize_transform = RandomResizedCrop(size=(384, 256))  # 무작위 크롭 및 리사이즈
                i, j, h, w = resize_transform.get_params(image, scale=(0.7, 1.0), ratio=(1, 1))
                image = TF.resized_crop(image, i, j, h, w, (384, 256))
            if random.random() < 0.5:
                image = RandFlip(1, 0)(image)  # 좌우 반전
                mask = RandFlip(1, 0)(mask)  # 좌우 반전
            if random.random() < 0.5:
                image, mask = random_rotation(image, mask)  # 무작위 회전

        mask[mask > 0] = 1  # 마스크 값을 이진화
        return image, mask, image_path  # 이미지, 마스크, 이미지 경로 반환

# 픽셀 정확도를 계산하는 함수.
def Pixel_Accuracy(yhat, ytrue, threshold=0.5):
    yhat = yhat > threshold  # 예측값을 이진화
    correct = torch.sum(yhat == ytrue)  # 예측이 실제값과 일치하는 픽셀 수 계산
    total = ytrue.numel()  # 총 픽셀 수 계산
    accuracy = correct.float() / total  # 정확도 계산
    return accuracy.item()

# Intersection over Union (IoU)를 계산하는 함수.
def Intersection_over_Union(yhat, ytrue, threshold=0.5):
    yhat = yhat > threshold  # 예측값을 이진화
    return IoU_Function(yhat, ytrue).nanmean().item()  # IoU 계산 및 반환

# Dice 계수를 계산하는 함수.
def Dice_Coefficient(yhat, ytrue, threshold=0.5):
    yhat = yhat > threshold  # 예측값을 이진화
    return Dice_Function()(yhat, ytrue).nanmean().item()  # Dice 계수 계산 및 반환

# 혼동 행렬을 계산하는 함수.
def Confusion_Matrix(yhat, ytrue, threshold=0.5):
    yhat = yhat > threshold  # 예측값을 이진화
    confusion_matrix = ConfusionMatrixMetric(metric_name=["recall", "precision", "f1 score"], reduction='mean', compute_sample=True)
    confusion_matrix(yhat, ytrue)  # 혼동 행렬 계산
    recall, precision, f1 = confusion_matrix.aggregate()  # 재현율, 정밀도, F1 점수 계산
    return recall, precision, f1

# 모델 학습을 위한 함수.
def train(train_loader, epoch, model, criterion, optimizer, device, max_iter):
    model.train()  # 모델을 학습 모드로 전환
    train_losses = AverageMeter()  # 손실 값을 저장할 객체 생성
    for i, (input, target, _) in enumerate(train_loader):
        iter_num = epoch * len(train_loader) + i
        poly_lr_scheduler(optimizer, backbone_lr, iter_num, max_iter)  # Poly learning rate scheduling 적용
        
        input = input.to(device)  # 입력을 GPU로 전송
        target = target.to(device)  # 타겟을 GPU로 전송
        output = model(input)  # 모델에 입력을 전달
        teacher_output = model(input).detach()  # 교사 모델의 출력을 얻음
        loss = criterion(output, teacher_output, target).float()  # 손실 계산
        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
        loss.backward()  # 역전파를 통해 기울기 계산
        optimizer.step()  # 옵티마이저를 통해 가중치 갱신
        train_losses.update(loss.detach().cpu().numpy(), input.shape[0])  # 손실 값을 업데이트
    Train_Loss = np.round(train_losses.avg, 6)  # 평균 손실 값을 반올림
    return Train_Loss

# 모델 검증을 위한 함수.
def validate(validation_loader, model, criterion, device, model_path=False, return_image_paths=False):
    if model_path != False:
        model.load_state_dict(torch.load(model_path))  # 모델 가중치를 로드
    model.eval()  # 모델을 평가 모드로 전환
    for i, (input, target, image_path) in enumerate(validation_loader):
        input = input.to(device)  # 입력을 GPU로 전송
        target = target.to(device)  # 타겟을 GPU로 전송
        with torch.no_grad():  # 기울기 계산 비활성화
            output = model(input)  # 모델에 입력을 전달
            teacher_output = model(input).detach()  # 교사 모델의 출력을 얻음
            loss = criterion(output, teacher_output, target).float()  # 손실 계산
        if i == 0:
            targets = target  # 첫 번째 배치의 타겟 저장
            outputs = output  # 첫 번째 배치의 출력 저장
            if return_image_paths == True:
                image_paths = image_path  # 이미지 경로 저장
        else:
            targets = torch.cat((targets, target))  # 이후 배치의 타겟을 연결
            outputs = torch.cat((outputs, output), axis=0)  # 이후 배치의 출력을 연결
            if return_image_paths == True:
                image_paths += image_path  # 이미지 경로를 연결
    if return_image_paths == True:
        return outputs, targets, image_paths  # 출력, 타겟, 이미지 경로 반환
    return outputs, targets  # 출력과 타겟 반환

# 클래스 이름을 문자열로 받아 해당 클래스 객체를 반환하는 함수.
def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

# 소스 파일을 출력 디렉토리로 복사하는 함수.
def copy_sourcefile(output_dir, src_dir='src'):
    import os
    import shutil
    import glob
    source_dir = os.path.join(output_dir, src_dir)

    os.makedirs(source_dir, exist_ok=True)  # 소스 디렉토리 생성
    org_files1 = os.path.join('./', '*.py')  # .py 파일 경로 설정
    org_files2 = os.path.join('./', '*.sh')  # .sh 파일 경로 설정
    org_files3 = os.path.join('./', '*.ipynb')  # .ipynb 파일 경로 설정
    org_files4 = os.path.join('./', '*.txt')  # .txt 파일 경로 설정
    org_files5 = os.path.join('./', '*.json')  # .json 파일 경로 설정
    files = []
    files = glob.glob(org_files1)
    files += glob.glob(org_files2)
    files += glob.glob(org_files3)
    files += glob.glob(org_files4)
    files += glob.glob(org_files5)

    # 소스 파일들을 출력 디렉토리로 복사
    tgt_files = os.path.join(source_dir, '.')
    for i, file in enumerate(files):
        shutil.copy(file, tgt_files)

# 학습 및 검증 손실 값을 저장하고 관리하는 클래스.
class LossSaver(object):
    def __init__(self):
        self.train_losses = []  # 학습 손실 값 저장 리스트
        self.val_losses = []  # 검증 손실 값 저장 리스트

    def reset(self):
        self.train_losses = []  # 학습 손실 값 초기화
        self.val_losses = []  # 검증 손실 값 초기화

    def update(self, train_loss, val_loss):
        self.train_losses.append(train_loss)  # 학습 손실 값 업데이트
        self.val_losses.append(val_loss)  # 검증 손실 값 업데이트

    def return_list(self):
        return self.train_losses, self.val_losses  # 학습 및 검증 손실 값 리스트 반환

    def save_as_csv(self, csv_file):
        df = pd.DataFrame({'Train Losses': self.train_losses, 'Validation Losses': self.val_losses})  # 손실 값들을 데이터프레임으로 저장
        df.index = [f"{i + 1} Epoch" for i in df.index]  # 인덱스를 에포크로 설정
        df.to_csv(csv_file, index=True)  # CSV 파일로 저장

# 평균 값을 계산하고 업데이트하는 클래스.
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0  # 현재 값 초기화
        self.avg = 0  # 평균 값 초기화
        self.sum = 0  # 합계 초기화
        self.count = 0  # 카운트 초기화

    def update(self, val, n=1):
        self.val = val  # 현재 값 업데이트
        self.sum += val * n  # 합계 업데이트
        self.count += n  # 카운트 업데이트
        self.avg = self.sum / self.count  # 평균 값 계산

# 이미지 경로와 마스크 경로를 수집하는 함수.
def collect_image_paths(base_dataset_dir, sub_dirs=['training', 'validation', 'test']):
    train_image_path_list = []  # 학습 이미지 경로 리스트
    train_target_path_list = []  # 학습 마스크 경로 리스트
    validation_image_path_list = []  # 검증 이미지 경로 리스트
    validation_target_path_list = []  # 검증 마스크 경로 리스트
    test_image_path_list = []  # 테스트 이미지 경로 리스트
    test_target_path_list = []  # 테스트 마스크 경로 리스트

    # 서브 디렉토리 이름과 리스트 매핑
    dir_to_lists = {
        'training': (train_image_path_list, train_target_path_list),
        'validation': (validation_image_path_list, validation_target_path_list),
        'test': (test_image_path_list, test_target_path_list)
    }

    for sub_dir in sub_dirs:
        full_dir_path = os.path.join(base_dataset_dir, sub_dir)  # 각 서브 디렉토리의 전체 경로를 생성

        image_list, target_list = dir_to_lists.get(sub_dir, (None, None))  # 해당 서브 디렉토리에 대한 이미지 및 마스크 리스트 선택

        if image_list is None or target_list is None:
            print(f"Unknown sub-directory: {sub_dir}")  # 알 수 없는 서브 디렉토리일 경우 경고 출력
            continue

        for file_name in os.listdir(full_dir_path):  # 서브 디렉토리 내의 파일들을 검색
            if (file_name.endswith(".png") or file_name.endswith(".jpg")) and "_mask" not in file_name:
                image_list.append(os.path.join(full_dir_path, file_name))  # 이미지 파일 경로 추가

                if file_name.endswith(".png"):
                    mask_file_name = file_name.replace(".png", "_mask.png")  # PNG 파일에 대응하는 마스크 이름 처리
                elif file_name.endswith(".jpg"):
                    mask_file_name = file_name.replace(".jpg", "_mask.jpg")  # JPG 파일에 대응하는 마스크 이름 처리

                target_list.append(os.path.join(full_dir_path, mask_file_name))  # 마스크 파일 경로 추가

    return (train_image_path_list, train_target_path_list,
            validation_image_path_list, validation_target_path_list,
            test_image_path_list, test_target_path_list)

# 실험을 수행하는 함수.
def Do_Experiment(iteration, model_name, model, train_loader, validation_loader, test_loader, Optimizer, lr, number_of_classes, epochs, Metrics, df, device, transform):
    start = timeit.default_timer()  # 실험 시작 시간을 기록
    train_bool = True
    test_bool = True
    
    criterion = CrossEntropyWithConsistencyLoss()  # Cross-Entropy와 Consistency Regularization을 포함한 손실 함수 설정
    
    # backbone과 segmentation head의 학습률을 다르게 적용하는 optimizer 설정
    optimizer = torch.optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': backbone_lr},
        {'params': model.segmentation_head.parameters(), 'lr': segmentation_head_lr}
    ], weight_decay=weight_decay)
    
    max_iter = epochs * len(train_loader)

    os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성
    control_random_seed(seed)  # 랜덤 시드 설정
    if train_bool:
        now = datetime.now()
        Train_date = now.strftime("%y%m%d_%H%M%S")  # 현재 시간 기록
        print('Training Start Time:', Train_date)
        best = 9999
        best_epoch = 1
        Early_Stop = 0
        loss_saver = LossSaver()  # 손실 값을 저장할 객체 생성
        train_start_time = timeit.default_timer()
        for epoch in range(1, epochs + 1):
            Train_Loss = train(train_loader, epoch, model, criterion, optimizer, device, max_iter)  # 한 에포크 동안 학습 수행
            outputs, targets = validate(validation_loader, model, criterion, device)  # 검증 수행
            Val_Loss = np.round(criterion(outputs, targets).cpu().numpy(), 6)  # 검증 손실 값 계산
            iou = np.round(Intersection_over_Union(outputs, targets), 3)  # IoU 계산
            dice = np.round(Dice_Coefficient(outputs, targets), 3)  # Dice 계수 계산
            now = datetime.now()
            date = now.strftime("%y%m%d_%H%M%S")
            print(str(epoch) + 'EP(' + date + '):', end=' ')
            print('T_Loss: ' + str(Train_Loss), end=' ')
            print('V_Loss: ' + str(Val_Loss), end=' ')
            print('IoU: ' + str(iou), end=' ')
            print('Dice: ' + str(dice), end=' ')

            loss_saver.update(Train_Loss, Val_Loss)  # 손실 값 업데이트
            loss_saver.save_as_csv(f'{output_dir}/Losses_{Experiments_Time}.csv')  # 손실 값을 CSV로 저장
            if Val_Loss < best:
                Early_Stop = 0
                torch.save(model.state_dict(), f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt')  # 모델 가중치 저장
                best_epoch = epoch  # 베스트 에포크 업데이트
                best = Val_Loss  # 베스트 손실 값 업데이트
                print('Best Epoch:', best_epoch, 'Loss:', Val_Loss)
            else:
                print('')
                Early_Stop += 1
            if Early_Stop >= EARLY_STOP:  # 얼리 스탑 조건 확인
                break
        train_stop_time = timeit.default_timer()  # 학습 종료 시간 기록
    if test_bool:
        now = datetime.now()
        date = now.strftime("%y%m%d_%H%M%S")
        print('Test Start Time:', date)
        outputs, targets, image_paths = validate(test_loader, model, criterion, device,
                                                 model_path=f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt',
                                                 return_image_paths=True)  # 테스트 수행
        Loss = np.round(criterion(outputs, targets).cpu().numpy(), 6)  # 테스트 손실 값 계산
        pa = np.round(Pixel_Accuracy(outputs.cpu(), targets.cpu()), 3)  # 픽셀 정확도 계산
        iou = np.round(Intersection_over_Union(outputs, targets), 3)  # IoU 계산
        dice = np.round(Dice_Coefficient(outputs, targets), 3)  # Dice 계수 계산
        recall, precision, f1 = Confusion_Matrix(outputs, targets)  # 혼동 행렬 계산
        recall = np.round(recall.cpu().numpy()[0], 3); precision = np.round(precision.cpu().numpy()[0], 3); f1 = np.round(f1.cpu().numpy()[0], 3);

        now = datetime.now()
        date = now.strftime("%y%m%d_%H%M%S")
        print('Best Epoch:', best_epoch)
        print('Test(' + date + '): ' + 'Loss: ' + str(Loss), end=' ')
        print('PA: ' + str(pa), end=' ')
        print('IoU: ' + str(iou), end=' ')
        print('Dice: ' + str(dice), end=' ')
        print('Recall: ' + str(recall), end=' ')
        print('Precision: ' + str(precision), end=' ')
        print('F1 Score: ' + str(f1), end='\n')

        stop = timeit.default_timer(); m, s = divmod((train_stop_time - train_start_time) / epoch, 60); h, m = divmod(m, 60); Time_per_Epoch = "%02d:%02d:%02d" % (h, m, s);
        m, s = divmod(stop - start, 60); h, m = divmod(m, 60); Time = "%02d:%02d:%02d" % (h, m, s);
        total_params = sum(p.numel() for p in model.parameters()); total_params = format(total_params, ',');
        Performances = [Experiments_Time, Train_date, iteration, model_name, best, Loss, pa, iou, dice, recall, precision, f1, total_params, Time, best_epoch, Time_per_Epoch, loss_function, lr, batch_size, epochs, FILENAME]
        df = df.append(pd.Series(Performances, index=df.columns), ignore_index=True)  # 실험 성능을 데이터프레임에 추가
        os.makedirs(f'{output_dir}/test_outputs', exist_ok=True)  # 출력 디렉토리 생성
        outputs = outputs.cpu().numpy()
        for output, image_path in zip(outputs, image_paths):
            np.save(f'{output_dir}/test_outputs/{os.path.basename(image_path)}', output)  # 출력 저장
    now = datetime.now()
    date = now.strftime("%y%m%d_%H%M%S")
    print('End', date)

    return df  # 데이터프레임 반환

In [52]:
# 현재 시간을 기록하고, 실험 시작 시간을 문자열로 변환하여 저장.
now = datetime.now()
Experiments_Time = now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:', Experiments_Time)  # 실험 시작 시간을 출력.

# 실험 성능을 저장할 데이터프레임의 열을 정의.
Metrics = ['Experiment Time', 'Train Time', 'Iteration', 'Model Name', 'Val_Loss', 
           'Test_Loss', 'PA', 'IoU', 'Dice', 'Recall', 'Precision', 'F1 Score', 
           'Total Params', 'Train-Prediction Time', 'Best Epoch', 'Time per Epoch', 
           'Loss Function', 'LR', 'Batch size', '#Epochs', 'DIR']
df = pd.DataFrame(index=None, columns=Metrics)  # 빈 데이터프레임을 생성.

# 실험 결과를 저장할 디렉토리를 생성.
output_root = f'output/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok=True)  # 출력 디렉토리를 생성.

# 설정된 반복 횟수만큼 반복.
for iteration in range(iterations[0], iterations[1] + 1):
    seed = iteration  # 현재 반복의 시드를 설정.
    control_random_seed(seed)  # 랜덤 시드를 설정.

    # 각 반복(iteration)에 맞는 데이터셋 분할 폴더를 설정.
    Dataset_dir = f'dataset/splits/pascal/1_8/split_{str(iteration).zfill(2)}'

    # 이미지 경로와 마스크 경로를 수집.
    (train_image_path_list, train_target_path_list,
     validation_image_path_list, validation_target_path_list,
     test_image_path_list, test_target_path_list) = collect_image_paths(Dataset_dir)

    # 학습, 검증, 테스트 데이터셋을 생성.
    train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, aug=True)
    validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list, aug=False)
    test_dataset = ImagesDataset(test_image_path_list, test_target_path_list, aug=False)
    
    # 데이터 로더를 생성.
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size,
        num_workers=0, pin_memory=True, shuffle=True,
    )
    validation_loader = torch.utils.data.DataLoader(
        validation_dataset, batch_size=batch_size,
        num_workers=0, pin_memory=True,
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size,
        num_workers=0, pin_memory=True,
    )

    # 지정된 모델 목록에 대해 반복.
    for model_name in model_names:
        print(f'{model_name} (Iter {iteration})')  # 현재 모델과 반복(iteration) 번호를 출력.
        output_dir = output_root + f'/{model_name}_Iter_{iteration}'  # 출력 디렉토리를 설정.
        copy_sourcefile(output_dir, src_dir='src')  # 소스 파일을 출력 디렉토리로 복사.
        control_random_seed(seed)  # 다시 랜덤 시드를 설정.
        
        # 모델 인스턴스를 생성.
        model = str_to_class(model_name)(in_channels, number_of_classes)
        device = torch.device("cuda:" + str(devices[0]))  # 사용할 GPU 장치를 설정.
        
        # 여러 GPU를 사용할 경우, DataParallel로 모델을 래핑.
        if len(devices) > 1:
            model = torch.nn.DataParallel(model, device_ids=devices).to(device)
        else:
            model = model.to(device)  # 단일 GPU를 사용할 경우, 모델을 GPU로 전송.

        # 실험을 수행하고, 결과를 데이터프레임에 저장.
        df = Do_Experiment(seed, model_name, model, train_loader, validation_loader, test_loader, 
                           optimizer, lr, number_of_classes, epochs, Metrics, df, device, None)
        
        # 실험 결과를 CSV 파일로 저장.
        try:
            df.to_csv(output_root + '/' + 'Semi_Seg' + Experiments_Time + '.csv', 
                      index=False, header=True, encoding="cp949")
        except:
            now = datetime.now()
            tmp_date = now.strftime("%y%m%d_%H%M%S")
            df.to_csv(output_root + '/' + 'Semi_Seg' + Experiments_Time + '_' + tmp_date + '_tmp' + '.csv', 
                      index=False, header=True, encoding="cp949")

# 실험 종료를 알리고, 프로그램을 종료.
import os
print('End')
os._exit(00)